In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

In [2]:
data_cleaned = pd.read_csv('cleaned_dataset.csv')

In [7]:
# Define features and target
X = data_cleaned[['NO2 AQI', 'O3 AQI', 'SO2 AQI', 'CO AQI']]
y = data_cleaned['Overall AQI']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the scalar
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# LinearRegression fit
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

# Make a prediction
y_pred = lr_model.predict(X_test_scaled)

# Evaluate the model
lr_mse = mean_absolute_error(y_test, y_pred)
lr_r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE) of Linear Regression: {lr_mse}")
print(f"R-Squared (R^2) of Linear Regression: {lr_r2}")

Mean Squared Error (MSE) of Linear Regression: 2.6053448774753054
R-Squared (R^2) of Linear Regression: 0.7914875994075485


In [20]:
# Random Forest Regressor
forest_model = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=4, min_samples_split=20)

# Train model
forest_model.fit(X_train, y_train)

# Make a prediction
y_pred_forest = forest_model.predict(X_test)

#Evaluate
forest_mse = mean_squared_error(y_test, y_pred_forest)
forest_r2 = r2_score(y_test, y_pred_forest)

print(f"Random Forest Regressor - Mean Squared Error: {forest_mse:.4f}")
print(f"Random Forest Regressor - R^2 Score: {forest_r2:.4f}")

Random Forest Regressor - Mean Squared Error: 3.0049
Random Forest Regressor - R^2 Score: 0.9483


In [ ]:
# Initialize SVR model
svr_model = SVR(kernel='rbf')

# Train model
svr_model.fit(X_train_scaled, y_train)

# Make a prediction
y_pred_svr = svr_model.predict(X_test_scaled)

# Evaluate
svr_mse = mean_squared_error(y_test, y_pred_svr)
svr_r2 = r2_score(y_test, y_pred_svr)

print(f"SVR - Mean Squared Error: {svr_mse:.4f}")
print(f"SVR - R^2 Score: {svr_r2:.4f}")